<a href="https://colab.research.google.com/github/AnoVando/Work2/blob/master/PIRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Richer syntax highlighting

Improved support for nested languages:

```notebook-python
df = pd.io.gbq.read_gbq('''
  SELECT 
    REGEXP_EXTRACT(name, '[a-zA-Z]+'),
    SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''')
```

In [0]:
import pandas as pd
import numpy as np
import glob
import os

In [0]:
# Materials
as400 = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400.csv")
as400_2 = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_2.csv")
cef = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/cef.csv")
copics = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/copics.csv")
dyna = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/dyna.csv")
kwaust = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/kwaust.csv")
winch = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/winch.csv")

as400.columns = ['Source', 'Product_ID', 'Description']
as400_2.columns = ['Source', 'Product_ID', 'Description']
cef.columns = ['Source', 'Product_ID', 'Description']
copics.columns = ['Source', 'Product_ID']
dyna.columns = ['Source', 'Product_ID', 'Description']
kwaust.columns = ['Source', 'Product_ID', 'Description']
winch.columns = ['Source', 'Product_ID', 'Description']

master_list = pd.concat([as400, as400_2, cef, copics, dyna, kwaust, winch], ignore_index=True, sort=False)
master_list.drop(columns='Description', inplace=True)
master_list.drop_duplicates(subset=['Source','Product_ID'], inplace=True)

In [3]:
master_list.groupby(["Source"])["Product_ID"].count()

Source
AS400     823638
copics    209899
dyna        9508
kwaust     16995
pec         1032
winch       5301
Name: Product_ID, dtype: int64

In [0]:
# PPD Supplier Part Relationship
as400_atl = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_atl.csv", encoding = 'ISO-8859-1')
as400_lan = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_lan.csv", encoding = 'ISO-8859-1')
as400_new = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_new.csv", encoding = 'ISO-8859-1')
as400_okc = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_okc.csv", encoding = 'ISO-8859-1')
as400_pty = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_pty.csv", encoding = 'ISO-8859-1')
as400_que = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_que.csv", encoding = 'ISO-8859-1')
as400_roc = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_roc.csv", encoding = 'ISO-8859-1')
as400_sea = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_sea.csv", encoding = 'ISO-8859-1')
as400_slp = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_slp.csv", encoding = 'ISO-8859-1')
as400_tor = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/as400_tor.csv", encoding = 'ISO-8859-1')

In [0]:
ppd_contracts_all = pd.concat([as400_atl, as400_lan, as400_new, as400_okc, as400_pty, as400_que, as400_roc, as400_sea,
                               as400_slp, as400_tor], ignore_index=True, sort=False)
ppd_contracts_all.columns = ['Product_ID', 'Plant', 'Supplier']

In [12]:
ppd_contracts_all.groupby(["Plant"])["Product_ID"].count()

Plant
CA51    512479
CA52    557475
MX51    358421
PA51    499942
US52    575337
US53    583747
US54    572305
US55    593191
US56    575572
US57    571717
Name: Product_ID, dtype: int64

In [15]:
as400_materials = pd.concat([as400, as400_2], ignore_index=True, sort=False)
as400_materials.drop(columns='Description', inplace=True)
as400_materials.drop_duplicates(subset=['Source','Product_ID'], inplace=True)
as400_materials.head()

,Source,Product_ID
0,AS400,SP1V57-11812
1,AS400,K028-2344
2,AS400,P551424
3,AS400,15-766DSP
4,AS400,SH8SU6807P11X


In [17]:
as400_pirs = as400_materials.merge(ppd_contracts_all, how='left', left_on = ['Product_ID'], right_on= ['Product_ID'])
as400_pirs['Supplier'].fillna('No Data', inplace=True)
as400_pirs.head()

,Source,Product_ID,Plant,Supplier
0,AS400,SP1V57-11812,US56,20495AA
1,AS400,SP1V57-11812,US54,20495AA
2,AS400,SP1V57-11812,US55,20495AA
3,AS400,SP1V57-11812,US57,20495AA
4,AS400,SP1V57-11812,PA51,20495AA


In [22]:
as400_pirs.groupby('Plant')['Product_ID'].count()

Plant
CA51    388845
CA52    402704
MX51    264141
PA51    373137
US52    416228
US53    422926
US54    413672
US55    429899
US56    416319
US57    413577
Name: Product_ID, dtype: int64

In [0]:
# Contracts
copics_contracts = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/contracts.csv", sep=',')
copics_contracts.columns = ['Supplier','Product_ID','Base_Price']
copics_contracts['Source'] = 'copics'
kwaust_contracts = pd.read_csv("https://raw.githubusercontent.com/AnoVando/Work2/master/kwaust_contracts.csv", sep=',')
kwaust_contracts.columns = ['Supplier','Product_ID','Base_Price']
kwaust_contracts['Source'] = 'kwaust'

contracts_all = pd.concat([copics_contracts, kwaust_contracts], ignore_index=True, sort=False)

In [0]:
contracts_all.head()

,Supplier,Product_ID,Base_Price,Source
0,05901AA,SA16095000,49.58,copics
1,05901AA,SA16095001,53.60,copics
2,05901AA,HP50034204,67.55,copics
3,05901AA,K113-81,49.35,copics
4,05901AA,K188-208,115.00,copics


In [0]:
contracts_all.describe()

,Base_Price
count,422929.000000
mean,1376.925845
std,3816.472284
min,0.000000
25%,18.340000
50%,210.450000
75%,595.000000
max,107862.120000


In [0]:
results = master_list.merge(contracts_all, how='left', left_on = ['Product_ID', 'Source'], right_on= ['Product_ID', 'Source'])
results['Supplier'].fillna('No Data', inplace=True)
results['Base_Price'].fillna(0, inplace=True)
results.head()

,Source,Product_ID,Supplier,Base_Price
0,AS400,SP1V57-11812,No Data,0.0
1,AS400,K028-2344,No Data,0.0
2,AS400,P551424,No Data,0.0
3,AS400,15-766DSP,No Data,0.0
4,AS400,SH8SU6807P11X,No Data,0.0


In [0]:
results.drop_duplicates(subset=['Supplier', 'Product_ID'], keep='last', inplace=True)

In [0]:
results.groupby(["Source"])['Product_ID'].count()

Source
AS400     797160
copics    233440
dyna        9334
kwaust     17025
pec         1032
winch       5301
Name: Product_ID, dtype: int64

In [0]:
from google.colab import files

results.to_excel('pir_data.xlsx')

In [0]:
as400_full = pd.concat([as400, as400_2], ignore_index=True, sort=False)
as400_full.drop_duplicates(subset=['Source','Product_ID'], inplace=True)

In [0]:
from google.colab import files

as400_full.to_excel('as400_full.xlsx')